# Table of Contents  

### 1. Introduction and background  

### 2. Methodology in brief

### 3. Detailed methodology and results  
#### &nbsp;&nbsp;&nbsp;&nbsp; 3.1. Data preprocessing  
#### &nbsp;&nbsp;&nbsp;&nbsp; 3.2. ARIMA-model fitting and forecasting  
#### &nbsp;&nbsp;&nbsp;&nbsp; 3.3. GARCH-model fitting and forecasting  

### 4. Performance Evaluation  
#### &nbsp;&nbsp;&nbsp;&nbsp; 4.1. ARIMA-model
#### &nbsp;&nbsp;&nbsp;&nbsp; 4.2. GARCH-model 

### 5. Use of LLMs  

### 6. Conclusion  

### 7. References  

# 1. Introduction and background 
The focus of this project is to research the development of the overall stock market in recent years, specifically the last 5 years. We choose to look at the S&P 500 index due to its good, altough not perfect, proxy for the development of the American economy. Therefore, the price levels of this index will be the research object of our project. The financial market exhibits strong time-series characteristics, including trends, seasonality, and volatility clustering, making it an ideal candidate for time-series modeling. Therefore, time-series models from empirical finance will be used to study historical trends and thereafter predict both price levels and volatility in the future. To predict price levels we will use an Autoregressive Integrated Moving Average model (ARIMA), and to predict volatility we will use a Generalized Autoregressive Conditional Heteroskedasticity model (GARCH).   

# 2. Methodology in brief

To complete this project we plan to use Python with libraries such as "statsmodels" for ARIMA estimation, "arch" for GARCH modeling, "matplotlib" and "seaborn" for data visualization, and "scikit-learn" for performance evaluation.

The price data on the S&P 500 index will be retrieved from Nasdaq.com and then undergo a preprocessing phase, where raw prices are converted into log returns to stabilize variance. Additionally, the data will be split into a training subset covering 256 weeks, and a test subset covering 4 weeks.

Then we will fit and select an ARIMA model for price level forecasting, by first checking for stationarity through the Dickey-Fuller test and then plotting the autocorrelation function (ACF) and partial autocorrelation function (PACF) to determine optimal model parameters. In a similar manner, we will fit a GARCH(1,1) model for volatility forecasting. Both models are run on the training data to forecast future prices and volatility for the S&P 500 index. These forecasts are visualized and compared to the actual data in the test set to evaluate the performance of both models. The ARIMA model is evaluated based on its Root Mean Squared Error (RMSE) and Mean Absolute Error (MAE) compared to the actual data, whereas the GARCH model performance is evaluated based on log-likelihood, AIC, and out-of-sample volatility forecasts using different volatility proxies.

Finally, the report will comment on the use of Language Learning Models (LLM) to solve this problem instead of time series models such as ARIMA and GARCH.

# 3. Detailed methodology and results  

### 3.1 Data Preprocessing  
The first phase of our project dealt with preprocessing the S&P 500 data. The daily closing prices of the index over the last 5 years was retrieved from XXX and converted into a csv file. Then the prices were converted into log returns, using the following formula:

$$r_t = \ln\left(\frac{P_t}{P_{t-1}}\right)$$

where r_t is the log return at time t and P_t is the closing price at time t.

Minor formatting changes were also completed before splitting the data into a training set of 256 weeks and a test set of 4 weeks which were each saved as separate csv files for later use. The details of our data preprocessing can be found in the commented code below.

In [1]:
#Import Relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import seaborn as sns
from arch import arch_model
from statsmodels.stats.diagnostic import het_arch, acorr_ljungbox
from sklearn.metrics import mean_squared_error, mean_absolute_error

#Load dataset
file_path = "S&P500_L5Y.csv"
df = pd.read_csv(file_path)

#Convert to datetime format and make sure its sorted in ascending order
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')
df = df.sort_values(by='Date')

#Rename 'Close/Last' column
df.rename(columns={'Close/Last': 'Close'}, inplace=True)

#Convert prices to log returns
df['Log_Returns'] = np.log(df['Close'] / df['Close'].shift(1))

#Drop first row due to conversion to log returns
df = df.dropna()

#Split data into training and testing subsets
train_size = 256 * 5  # 1280 business days = 256 weeks
test_size = 4 * 5  # 20 business days = 4 weeks
train_data = df.iloc[:-test_size]
test_data = df.iloc[-test_size:]

#Save as csv-files
train_data.to_csv("train_data.csv", index=False)
test_data.to_csv("test_data.csv", index=False)

ModuleNotFoundError: No module named 'statsmodels'

Next, we plot Log-returns to visualize data

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df.index, df["Log_Returns"], label="Log Returns", color="blue")
plt.axhline(y=0, color="black", linestyle="--", linewidth=0.8)
plt.xlabel("Date")
plt.ylabel("Log Returns")
plt.title("Log Returns Over Time")
plt.legend()
plt.gcf().autofmt_xdate()  # Optional: nicely format the date labels
plt.show()


We can see that the log-returns in the beginning of 2020 were volatile, which is due market uncertainty related to the Covid pandemic

### 3.2 ARIMA-model fitting and forecasting  

We will use an ARIMA model to forecast price levels of the S&P 500. We load the preprocessed training data and index it by date to faciliate for time-series analysis. Then we perform a Dickey-Fuller test to check if the data is stationary. This is important because the ARIMA-model assumes that a time series is stationary, meaning that statistical properties such as mean and varaince should not change over time. If the resulting p-value from the test is greater than 0.05, the data is categorized as non-stationary, meaning that it must be differenced before it can be applied in the ARIMA model. Differencing involves subtracting the previous value from the current value to remove trends and stabilize the mean. On the other hand, if the resulting p-value from the test is below 0.05, the data is already stationary and we can use it directly in the ARIMA model. The code below checks for stationarity and completes differencing if necessary. It then outputs a short description of what is has done.

In [ ]:
#Import relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
from sklearn.metrics import mean_absolute_error, mean_squared_error

#Load preprocessed training data
df = pd.read_csv("train_data.csv")
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

#Test stationarity
adf_test = adfuller(df['Close'])
if adf_test[1] > 0.05:  # If p-value > 0.05, the series is non-stationary
    d = 1
else:
    d = 0

#Print results, applying differencing if necessary
if d == 1:
    df['Close_diff'] = df['Close'].diff().dropna()
    data_to_use = df['Close_diff'].dropna()
    print("The data was non-statiornary. Differencing has been completed")
else:
    data_to_use = df['Close']
    print("The data was already statiornary. No differencing necessary")

Now that we have ensured stationary data we can plot the ACF and PACF in order to determine optimal model paramters. The ACF measures how much past values influence future values by showing the correlation between our time series and different lagged values. At the lag where the ACF plot dies down, we can determine the moving average component (q) of our process. The PACF is similar, but measures direct correlation to a specific lag, removing the effect of other lags in between. At the lag where the PACF plot dies down, we can determine the autoregressive component (p) of our process. Both plots are outputted by the code below, which first outputs the normal ACF and PACF and then outputs versions that have removed lag 0, since this always has a correlation equal to 1 anyaways, and makes it harder to read the other results. 

In [ ]:
from statsmodels.tsa.stattools import acf, pacf

#Plot ACF and PACF to determine p and q
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
plot_acf(data_to_use, ax=axes[0]) 
plot_pacf(data_to_use, ax=axes[1]) 
plt.show()

#Compute ACF and PACF values (excluding lag 0)
acf_values = acf(data_to_use, nlags=40)[1:] 
pacf_values = pacf(data_to_use, nlags=40)[1:] 
lags = np.arange(1, 41)
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Plot ACF without lag 0
axes[0].bar(lags, acf_values, width=0.3, color='blue')
axes[0].set_title("ACF (Lag 0 Removed)")
axes[0].set_xlabel("Lag")
axes[0].set_ylabel("Autocorrelation")

# Plot PACF without lag 0
axes[1].bar(lags, pacf_values, width=0.3, color='red')
axes[1].set_title("PACF (Lag 0 Removed)")
axes[1].set_xlabel("Lag")
axes[1].set_ylabel("Partial Autocorrelation")

plt.show()


From the plots above we can see that correlation dies out over time, but even with lag 0 removed it is hard to make out exactly where the correlation dies out sufficiently to determine q and p. Therefore, we use the built-in function "auto_arima" which systematically tries out different parameter combinations and uses the Akaike Information Criterion (AIC) to select the best model. AIC balances goodness of fit with model complexity to prevent overfitting. The optimal ARIMA order is finally outputted at the end of the code below.

In [ ]:
#Built-in function to determine paramters
auto_model = auto_arima(df['Close'], seasonal=False, stepwise=True, trace=True, suppress_warnings=True)
p, d, q = auto_model.order
print(f"Optimal ARIMA Order: ({p}, {d}, {q})")

Now that the model parameters are determined as q = 3 and p = 3, we can fit the model to the dataset. A summary of the fitted model is outputted below.

In [ ]:
#Fit model
model = ARIMA(df['Close'], order=(p, d, q))
model_fit = model.fit()
print(model_fit.summary())

From the outputted model above we can see that the autoregressive and moving average coefficients all have large standard errors that lead to high p-values. This suggests that the coefficients are not really statistically significant and therefore we could expect that the model will not perform very well in predicting future prices of the S&P 500 index which we will complete below. 

The test for constant variance with H = 0.79 suggests some heteroskedasticity, meaning that the variance of the residuals changes over time. This indicates that a GARCH model could be suitable to better capture volatility, which we will complete in section 3.3.

The code below loads the test dataset and uses the fitted ARIMA model to forecast future price levels. It also calulates the RMSE and MAE of the model. The forecast is plotted together with the actual price data, and the results will be further discussed under "Performance Evaluation" in section 4.1

In [ ]:
#Load test dataset
test_data = pd.read_csv("test_data.csv")
test_data['Date'] = pd.to_datetime(test_data['Date'])
test_data.set_index('Date', inplace=True)

#Forecast future price levels
forecast_obj = model_fit.get_forecast(steps=len(test_data))
forecast_mean = forecast_obj.predicted_mean
forecast_conf_int = forecast_obj.conf_int()

#Plot actual vs. predicted prices
plt.figure(figsize=(10, 5))
plt.plot(test_data.index, test_data['Close'], label="Actual Prices", marker='o')
plt.plot(test_data.index, forecast_mean, label="Predicted Prices", linestyle='dashed', marker='x')
plt.fill_between(test_data.index, 
                 forecast_conf_int.iloc[:, 0], 
                 forecast_conf_int.iloc[:, 1], 
                 color='gray', alpha=0.3, label="95% Confidence Interval")
plt.legend()
plt.title(f"ARIMA({p},{d},{q}) Forecast vs Actual Prices")
plt.show()

#Evaluate the model
rmse = np.sqrt(mean_squared_error(test_data['Close'], forecast_mean))
mae = mean_absolute_error(test_data['Close'], forecast_mean)
print(f"RMSE: {rmse:.5f}")
print(f"MAE: {mae:.5f}")

### 3.3 GARCH-model fitting and forecasting  

We will use a GARCH model to forecast volatility levels of the S&P 500. First, we perform an ARCH-LM test to detect whether the time series exhibits time-varying volatility, in which case a GARCH model is likely to capture the clustering in volatility. If the resulting p-value is below our threshold of 5%, then the null hypothesis (that there are no ARCH effects) is rejected.

In [ ]:
# Arch-LM test

lm_test = het_arch(train_data['Log_Returns'], nlags=10) 
print("\nARCH-LM Test on Log_Returns:") 
print(f"LM Statistic: {lm_test[0]:.4f}") 
print(f"LM Test p-value: {lm_test[1]:.4f}") 
print(f"F-statistic: {lm_test[2]:.4f}") 
print(f"F-test p-value: {lm_test[3]:.4f}")


Since the p-value is below 5%, we reject the null hypothesis, indicating that a GARCH model is appropriate.

We then test different GARCH configurations:

In [ ]:
results = []
orders = [(1, 1), (1, 2), (2, 1)]
distributions = ['normal', 't', 'skewt']

train_data = train_data["Log_Returns"]

# Loop over each candidate configuration
for p, q in orders:
    for dist in distributions:
        try:
            model = arch_model(100 * train_data, vol='GARCH', p=p, q=q, dist=dist, rescale=False)
            res = model.fit(disp='off')
            results.append({
                'order': f"GARCH({p},{q})",
                'distribution': dist,
                'AIC': res.aic,
                'BIC': res.bic,
                'LogLikelihood': res.loglikelihood
            })
            print(f"Fitted GARCH({p},{q}) with {dist} distribution: AIC={res.aic:.2f}, BIC={res.bic:.2f}")
        except Exception as e:
            print(f"Model GARCH({p},{q}) with {dist} failed: {e}")

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)
results_df

GARCH(1,1) is selected because it is both the simplest and the best-performing specification for capturing volatility clustering in this case. 

Now we check the residual diagnostics to ensure no significant autocorrelation remains in the standardized residuals.

In [ ]:
std_resid = res.std_resid

plt.figure(figsize=(10, 4)) 
plt.plot(std_resid) 
plt.title("Standardized Residuals") 
plt.xlabel("Time") 
plt.ylabel("Standardized Residual") 
plt.show()

plt.figure(figsize=(10, 4)) 
plot_acf(std_resid, lags=20, alpha=0.05) 
plt.title("ACF of Standardized Residuals") 
plt.show()

plt.figure(figsize=(10, 4)) 
plot_acf(std_resid**2, lags=20, alpha=0.05) 
plt.title("ACF of Squared Standardized Residuals") 
plt.show()

plt.figure(figsize=(6, 6)) 
sm.qqplot(std_resid, line='s') 
plt.title("QQ Plot of Standardized Residuals") 
plt.show()

lb_test = acorr_ljungbox(std_resid**2, lags=[10], return_df=True) 
print("Ljung-Box test on squared standardized residuals (lag 10):") 
print(lb_test)

A high p-value for the Ljung-Box test supports the null hypothesis of no further serial correlation in the squared residuals, suggesting that the model captures volatility clustering adequately.

Next, we use a GARCH (1,1) to forecast volatility on the test set:

In [ ]:
garch_model = arch_model(100 * train_data, vol='GARCH', p=1, q=1, dist='skewt', rescale=False)
garch_result = garch_model.fit(disp='off')
print(garch_result.summary())

forecast_horizon = len(test_data)
forecasts = garch_result.forecast(horizon=forecast_horizon, reindex=False)
forecasted_variance = forecasts.variance.iloc[-1]
forecasted_volatility = np.sqrt(forecasted_variance)

forecast_var_series = pd.Series(forecasted_variance.values, index=test_data.index)
forecast_vol_series = pd.Series(forecasted_volatility.values, index=test_data.index)

We compare the GARCH model’s forecasted volatility to two proxies: rolling realized volatility and squared absolute returns. The rolling realized volatility is calculated as the square root of the average squared returns over a fixed window (e.g., 5 days), smoothing out daily noise. In contrast, the squared absolute returns proxy captures immediate, day-to-day volatility shocks. Using both proxies lets us assess the model's performance in reflecting long-term trends versus short-term fluctuations.

In [ ]:
window = 5
# Combine returns ensuring consistent indices (train_data is assumed to be a Series)
combined_returns = pd.concat([train_data.tail(window-1), test_data["Log_Returns"]])
combined_sq = (100 * combined_returns)**2
combined_roll_mean = combined_sq.rolling(window=window).mean()
realized_var_roll = combined_roll_mean.loc[test_data.index]
realized_vol_roll = np.sqrt(realized_var_roll)
proxy_var = (100 * np.abs(test_data["Log_Returns"]))**2

# Plot forecasted volatility vs. rolling realized volatility
plt.figure()
plt.plot(test_data.index, forecast_vol_series, marker='o', linestyle='-', label='Forecasted Volatility')
plt.plot(test_data.index, realized_vol_roll, marker='x', linestyle='--', label=f'{window}-Day Rolling Realized Volatility')
plt.title("Forecasted Volatility vs. Rolling Realized Volatility")
plt.xlabel("Date")
plt.ylabel("Volatility")
plt.legend()
plt.show()

# Plot forecasted volatility (variance) vs. squared absolute returns proxy
plt.figure()
plt.plot(test_data.index, forecast_var_series, marker='o', linestyle='-', label='Forecasted Volatility')
plt.plot(test_data.index, proxy_var, marker='x', linestyle='--', label='Squared Absolute Returns (Proxy)')
plt.title("Forecasted Volatility vs. Squared Absolute Returns (Proxy)")
plt.xlabel("Date")
plt.ylabel("Volatility / Squared Returns")
plt.legend()
plt.show()

# Evaluation for Rolling Realized Volatility Proxy
common_index = realized_vol_roll.dropna().index.intersection(forecast_vol_series.index)
realized_vol = realized_vol_roll.loc[common_index]
predicted_vol = forecast_vol_series.loc[common_index]

mse_vol = mean_squared_error(realized_vol, predicted_vol)
rmse_vol = np.sqrt(mse_vol)
mae_vol = mean_absolute_error(realized_vol, predicted_vol)
realized_var = realized_vol**2
predicted_var = forecast_var_series.loc[common_index]
qlike_vol = np.mean(np.log(predicted_var) + (realized_var / predicted_var))

print("Volatility Forecast Evaluation (Rolling Realized Volatility Proxy):")
print(f" MSE:   {mse_vol:.4f}")
print(f" RMSE:  {rmse_vol:.4f}")
print(f" MAE:   {mae_vol:.4f}")
print(f" QLIKE: {qlike_vol:.4f}")

# Calculate average realized volatility over the evaluation period
avg_realized_vol = realized_vol.mean()
rmse_vol_pct = (rmse_vol / avg_realized_vol) * 100
mae_vol_pct = (mae_vol / avg_realized_vol) * 100

print("\nPercentage Error Metrics (Rolling Realized Volatility Proxy):")
print(f" RMSE (% of avg vol):  {rmse_vol_pct:.2f}%")
print(f" MAE (% of avg vol):   {mae_vol_pct:.2f}%")

# Evaluation for Squared Absolute Returns Proxy
common_index_proxy = proxy_var.dropna().index.intersection(forecast_var_series.index)
mse_proxy = mean_squared_error(proxy_var.loc[common_index_proxy], forecast_var_series.loc[common_index_proxy])
rmse_proxy = np.sqrt(mse_proxy)
mae_proxy = mean_absolute_error(proxy_var.loc[common_index_proxy], forecast_var_series.loc[common_index_proxy])
qlike_proxy = np.mean(np.log(forecast_var_series.loc[common_index_proxy]) + (proxy_var.loc[common_index_proxy] / forecast_var_series.loc[common_index_proxy]))

print("\nVolatility Forecast Evaluation (Squared Absolute Returns Proxy):")
print(f" MSE:   {mse_proxy:.4f}")
print(f" RMSE:  {rmse_proxy:.4f}")
print(f" MAE:   {mae_proxy:.4f}")
print(f" QLIKE: {qlike_proxy:.4f}")

# For the proxy, compute an average volatility from the squared returns
avg_proxy_vol = np.sqrt(proxy_var.loc[common_index_proxy].mean())
rmse_proxy_pct = (rmse_proxy / avg_proxy_vol) * 100
mae_proxy_pct = (mae_proxy / avg_proxy_vol) * 100

print("\nPercentage Error Metrics (Squared Absolute Returns Proxy):")
print(f" RMSE (% of avg vol):  {rmse_proxy_pct:.2f}%")
print(f" MAE (% of avg vol):   {mae_proxy_pct:.2f}%")


# 4. Performance Evaluation  

### 4.1 ARIMA-model  
The price levels predicted by the ARIMA model in section 3.2 are relatively flat compared to the actual prices which fluctuate. This may suggest the need for a more complex model. Also, the 95% confidence interval is quite wide, ranging almost 800 price points, which means that there is quite a bit of uncertainty in the predictions. A RMSE of 108.32 and MAE of 86.56 may seem high, but considering that the prices of the index are in the scale of 6000 these numbers indicate roughly a difference of 1.8% and 1.4% respectively to actual prices. 

### 4.2 GARCH-model Evaluation

The GARCH(1,1) model appears to reasonably capture the general patterns of time-varying volatility in the S&P 500 returns.

The volatility forcasting plots reveal some differences between volatility proxies. The 5-day rolling realized volatility is smoother and gradually reflects changes, as it averages squared returns over several days, capturing persistent volatility patterns. In contrast, the squared absolute returns proxy is more uneven, reflecting immediate day-to-day fluctuations.

The model's performance varied across the different proxies. This variation is likely due to the same reason mentioned earlier: the 5-day rolling realized volatility is inherently smoother, which aligns better with the stable predictions produced by the GARCH model. 

**Results of best performing proxy, which is Rolling Realized Volatility:**  
- **MSE:** 0.085  
- **RMSE:** 0.292 (approximately 36.3% relative to average volatility)  
- **MAE:** 0.257 (approximately 31.9% relative to average volatility)  
- **QLIKE:** 0.668  

However, as evidenced by the results using the absolute returns proxy, our model struggled to capture day-to-day volatility shocks. If capturing such high-frequency fluctuations were the primary goal, a model specifically designed to accommodate abrupt changes in volatility would be more appropriate.

# 5. Use of LLMs 

We used LLM's like ChatGPT for grammar checking, code debugging and a second opinion when discussing results.  

# 6. Conclusion 

The ARIMA model offers forecasts that performed well relative to the index's scale, but its flat predictions and wide confidence intervals indicate that a more sophisticated approach may be needed for price levels. The GARCH(1,1) model seems to capture persistent volatility patterns when compared against a 5-day rolling realized volatility proxy. However, its performance drops when evaluated with the more volatile squared absolute returns proxy, highlighting its limitations in capturing day-to-day shocks. Overall, while these models provide useful insights into long-term trends, alternative methods might be required for high-frequency volatility forecasting.

# 7. References

- Brooks, C (2019). Introductory Econometrics for Finance
- Risstad, M. (2025). Univariate time-series models | Forecasting (Lecture slides). TIØ4317 – Empirical and Quantitative Methods in Finance.
- Risstad, M. (2025). Univariate GARCH models (Lecture slides). TIØ4317 – Empirical and Quantitative Methods in Finance.